In [1]:
from sklearn.metrics.pairwise import pairwise_distances
import tensorflow as tf
import numpy as np
import time
import h5py
import sys

In [3]:
# Resnet blocks 
def identity_block(input_tensor, kernel_size, filters, stage, block, is_training):
    with tf.variable_scope("stage-{}_block-{}".format(stage, block), reuse=tf.AUTO_REUSE) as scope:
        x = tf.layers.batch_normalization(input_tensor, training=is_training)
        x = tf.nn.relu(x)
        x = tf.layers.conv2d(x, filters[0], kernel_size[0], padding="same")

        x = tf.layers.batch_normalization(x, training=is_training)
        x = tf.nn.relu(x)
        x = tf.layers.conv2d(x, filters[1], kernel_size[1], padding="same")

        x = tf.concat([x, input_tensor], axis=-1)
    return x

def conv_block(input_tensor, kernel_size, filters, stage, block, is_training, strides=2):
    with tf.variable_scope("stage-{}_block-{}".format(stage, block), reuse=tf.AUTO_REUSE) as scope:
        x = tf.layers.batch_normalization(input_tensor, training=is_training)
        x = tf.nn.relu(x)
        x = tf.layers.conv2d(x, filters[0], kernel_size[0], strides=strides, padding="same")

        x = tf.layers.batch_normalization(x, training=is_training)
        x = tf.nn.relu(x)
        x = tf.layers.conv2d(x, filters[1], kernel_size[1], padding="same")

        shortcut = tf.layers.batch_normalization(input_tensor, training=is_training)
        shortcut = tf.nn.relu(shortcut)
        shortcut = tf.layers.conv2d(shortcut, filters[1], kernel_size[1], strides=strides, padding="same")

        x = tf.concat([x, shortcut], axis=-1)
    return x

# slimmed downed resnet
def classifier(inputs, num_classes, num_embeddings, is_training):
    with tf.variable_scope("stage-1_block-a", reuse=tf.AUTO_REUSE) as scope:
        x = tf.layers.conv2d(inputs, 64, 7, strides=2, padding="same")
        x = tf.layers.batch_normalization(x, training=is_training)
        x = tf.nn.relu(x)
        x = tf.layers.max_pooling2d(x, 3, strides=2, padding="same")

    x = conv_block    (x, [1, 3], [64, 256], stage=2, block='a', is_training=is_training)
    x = identity_block(x, [1, 3], [64, 256], stage=2, block='b', is_training=is_training)

    x = conv_block    (x, [1, 3], [128, 512], stage=3, block='a', is_training=is_training)
    x = identity_block(x, [1, 3], [128, 512], stage=3, block='b', is_training=is_training)

    x = conv_block    (x, [1, 3], [256, 1024], stage=4, block='a', is_training=is_training)
    x = identity_block(x, [1, 3], [256, 1024], stage=4, block='b', is_training=is_training)

    x = conv_block    (x, [1, 3], [512, 2048], stage=5, block='a', is_training=is_training)
    x = identity_block(x, [1, 3], [512, 2048], stage=5, block='b', is_training=is_training)

    with tf.variable_scope("embeddings", reuse=tf.AUTO_REUSE) as scope:
        x = tf.layers.average_pooling2d(x, (x.get_shape()[-3], x.get_shape()[-2]), 1) #global average pooling
        x = tf.layers.flatten(x)
        embeddings = tf.layers.dense(x, num_embeddings, activation=None)
        
    with tf.variable_scope("logits", reuse=tf.AUTO_REUSE) as scope:
        pred_logits = tf.nn.relu(embeddings)
        pred_logits = tf.layers.dense(pred_logits, num_classes, activation=None)

    return embeddings, pred_logits

In [4]:
# Center Loss (Metric Learning)
# https://github.com/EncodeTS/TensorFlow_Center_Loss/blob/master/center_loss.py
def get_center_loss(features, labels, alpha, num_classes):
    
    len_features = features.get_shape()[1] 
    with tf.variable_scope("central_loss", reuse=tf.AUTO_REUSE) as scope:
        centers = tf.get_variable('centers', [num_classes, len_features], dtype=tf.float32,
            initializer=tf.constant_initializer(0), trainable=False)
        
    labels = tf.reshape(labels, [-1])

    centers_batch = tf.gather(centers, labels)
    loss = tf.nn.l2_loss(features - centers_batch)

    unique_label, unique_idx, unique_count = tf.unique_with_counts(labels)
    appear_times = tf.gather(unique_count, unique_idx)
    appear_times = tf.reshape(appear_times, [-1, 1])

    diff = centers_batch - features
    diff = diff / tf.cast((1 + appear_times), tf.float32)
    diff = alpha * diff

    centers_update_op = tf.scatter_sub(centers, labels, diff)
    tf.add_to_collection(tf.GraphKeys.UPDATE_OPS, centers_update_op)

    return loss, centers

In [5]:
def unison_shuffled_copies(a, b):
    p = np.random.permutation(a.shape[0])
    return a[p], b[p]

def get_batch(data_x, data_y, batch_size):
    while True:
        data_x, data_y = unison_shuffled_copies(data_x, data_y)
        for index in range(0, data_x.shape[0], batch_size):
            x, y = data_x[index:index+batch_size], data_y[index:index+batch_size]
            if x.shape[0] == batch_size:
                yield x, y

In [22]:
# define model parameters and load dataset
data_file =    "data/CSI_50_500.h5"
weights_path = "data/weights/central_loss_50_500/central_loss_model.ckpt"

num_embeddings = 64
num_classes = 50
batch_size = 8
ratio = 0.848
alpha = 0.5
    
hf = h5py.File(data_file, 'r')
X_train = np.expand_dims(hf.get('X_train'), axis=-1)
X_test = np.expand_dims(hf.get('X_test'), axis=-1)
y_train = np.eye(num_classes)[hf.get('y_train')]
y_test = np.eye(num_classes)[hf.get('y_test')]
y_train_n = np.array(hf.get('y_train'))
y_test_n = np.array(hf.get('y_test'))
train_classes = np.array(hf.get('labels')).astype(str)
X_data = np.expand_dims(hf.get('X_intruder'), axis=-1)
y_data = np.array(hf.get('y_intruder'))
hf.close()

rows, cols, channels = X_train.shape[1:]

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape, X_data.shape, y_data.shape, "\n")

(1359, 2000, 270, 1) (1359, 50) (240, 2000, 270, 1) (240, 50) (400, 2000, 270, 1) (400,) 



In [23]:
# method to get model predictions
tf.reset_default_graph()

with tf.device('/cpu:0'):
    with tf.variable_scope("Inputs") as scope:
        X = tf.placeholder(tf.float32, [None, rows, cols, channels])
        Y = tf.placeholder(tf.float32, [None, num_classes])
        
    is_training = tf.placeholder(tf.bool)

    # Calculate the gradients for each model tower.
    with tf.device("/gpu:0"):
        with tf.name_scope("resnet_0") as scope:
            embeddings, pred_logits = classifier(X, num_classes, num_embeddings, is_training)

            center_loss, centers = get_center_loss(embeddings, tf.argmax(Y, axis=1), alpha, num_classes)
    # Initializing the variables
    init = tf.global_variables_initializer()
    merged = tf.summary.merge_all()
    saver = tf.train.Saver(max_to_keep=5)

In [24]:
# get model predictions
X_data_embeddings = []
X_test_embeddings = []
X_data_logits = []
X_test_logits = []
X_train_embeddings = []
X_train_logits = []
final_centers = []

with tf.Session(config=tf.ConfigProto(allow_soft_placement = True)) as sess:
    sess.run(init)
    saver.restore(sess, tf.train.latest_checkpoint(weights_path))
    
    final_centers = sess.run([centers], feed_dict={is_training: False})

    for step in range(X_data.shape[0]):
        tmp1, tmp2 = sess.run([embeddings, pred_logits], feed_dict={X: np.expand_dims(X_data[step], axis=0), is_training: False})
        X_data_embeddings.append(tmp1)
        X_data_logits.append(tmp2)

    for step in range(X_test.shape[0]):
        tmp1, tmp2 = sess.run([embeddings, pred_logits], feed_dict={X: np.expand_dims(X_test[step], axis=0), is_training: False})
        X_test_embeddings.append(tmp1)
        X_test_logits.append(tmp2)
        
    for step in range(X_train.shape[0]):
        tmp1, tmp2 = sess.run([embeddings, pred_logits], feed_dict={X: np.expand_dims(X_train[step], axis=0), is_training: False})
        X_train_embeddings.append(tmp1)
        X_train_logits.append(tmp2)

X_test_embeddings = np.squeeze(X_test_embeddings)
X_data_embeddings = np.squeeze(X_data_embeddings)
X_train_embeddings = np.squeeze(X_train_embeddings)
X_train_logits = np.squeeze(X_train_logits)
X_test_logits = np.squeeze(X_test_logits)
X_data_logits = np.squeeze(X_data_logits)
final_centers = np.squeeze(final_centers)

print(X_data_logits.shape, X_test_logits.shape, X_train_logits.shape, X_data_embeddings.shape, X_test_embeddings.shape, X_train_embeddings.shape, final_centers.shape)

INFO:tensorflow:Restoring parameters from /home/kjakkala/neuralwave/data/weights/central_loss_mc/mc_0/central_loss_model.ckpt-10500
(400, 50) (240, 50) (1359, 50) (400, 64) (240, 64) (1359, 64) (50, 64)


In [25]:
print(np.mean(np.equal(np.argmax(X_train_logits, axis=-1), np.argmax(y_train, axis=-1))))
print(np.mean(np.equal(np.argmax(X_test_logits, axis=-1), np.argmax(y_test, axis=-1))))

0.03899926416482708
0.020833333333333332


In [32]:
# predicts intruders or classes not present in the training dataset by thresholding the data embeddings
def get_preds(data):
    class_thresholds = []
    for i in range(num_classes):
        class_thresholds.append(np.max(np.min(pairwise_distances(X_train_embeddings[np.where(y_train_n == i)], final_centers), axis=1)))
    class_thresholds = np.array(class_thresholds)
    class_thresholds += class_thresholds*2.5

    distance = np.min(pairwise_distances(data, final_centers), axis=1)
    class_preds = np.argmin(pairwise_distances(data, final_centers), axis=1)

    for i in range(data.shape[0]):
        if (distance[i] > class_thresholds[class_preds[i]]):
            class_preds[i] = -1
            
    return class_preds, distance

print(np.mean(np.equal(get_preds(X_train_embeddings)[0], y_train_n))*100)
print(np.mean(np.equal(get_preds(X_test_embeddings)[0], y_test_n))*100)
print(np.mean(np.equal(get_preds(X_data_embeddings)[0], -1*np.ones(X_data_embeddings.shape[0])))*100)

100.0
90.0
73.0


In [22]:
intruders = get_preds(X_data_embeddings)[0]
for i in range(len(intruders)):
    print("{: <5}\t{: <30}".format(train_classes[intruders[i]], data_classes[y_data[i]]))

wei  	benjamin_poole                
ding 	benjamin_poole                
wei  	benjamin_poole                
wei  	benjamin_poole                
wei  	benjamin_poole                
mark_powers	benjamin_poole                
wei  	benjamin_poole                
wei  	benjamin_poole                
wei  	benjamin_poole                
ding 	benjamin_poole                
wei  	benjamin_poole                
ding 	benjamin_poole                
wei  	benjamin_poole                
ding 	benjamin_poole                
wei  	benjamin_poole                
ding 	benjamin_poole                
wei  	benjamin_poole                
wei  	benjamin_poole                
ding 	benjamin_poole                
wei  	benjamin_poole                
mark_powers	benjamin_poole                
wei  	benjamin_poole                
wei  	benjamin_poole                
wei  	benjamin_poole                
wei  	benjamin_poole                
ding 	benjamin_poole                
wei  	benjamin_poole      